In [ ]:
import datetime as dt
from viresclient import SwarmRequest
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

%load_ext autoreload
%autoreload 2
import utils

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,cartopy

In [ ]:
# Identify recent 7-day time window to use
today = dt.datetime.combine(dt.date.today(), dt.time())
three_days_ago = today - dt.timedelta(days=3)
ten_days_ago = today - dt.timedelta(days=10)
start, end = ten_days_ago, three_days_ago
start, end

In [ ]:
# Fetch data
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    sampling_step="PT1M"
)
data = request.get_between(start, end)
ds = data.as_xarray()
ds

In [ ]:
# Build figure to plot onto
fig = plt.figure(figsize=(10, 5))
ax_Moll = plt.subplot2grid(
    (1, 1), (0, 0), colspan=1,
    projection=ccrs.Mollweide()
)
ax_Moll.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.5)

# Draw on the measurements
ds.plot.scatter(
    "Longitude", "Latitude", "F",
    ax=ax_Moll, transform=ccrs.PlateCarree(),
    s=10, alpha=0.4, marker=".",
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Evaluate model and draw contours
coords = utils.grid(height=430)
CHAOS_Core = utils.eval_model(time=start,coords=coords)
utils.plot_contours(
    ax_Moll,
    coords[..., 1], coords[..., 0], CHAOS_Core["F"],
    draw_labels=False, linewidths=2, levels=20, alpha=0.5,
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Prepare descriptive text
ID2NAME = {"A": "ALPHA", "B": "BRAVO", "C": "CHARLIE"}
spacecraft = ID2NAME.get(ds['Spacecraft'].values[0])
spacecraft = f"SWARM {spacecraft}"
altitude = int(round((ds["Radius"].values[0] - 6371200)/1e3, ndigits=-1))
quantity = "Magnetic field intensity (F)"
sample_rate = "1 Minute"
description = f"""
Spacecraft:   {spacecraft} (~{altitude}km altitude)
Time period:  {start.strftime('%Y-%m-%d')} to {(end-dt.timedelta(days=1)).strftime('%Y-%m-%d')}
Quantity:     {quantity}
Downsampling: {sample_rate}
[Dots: Measurements, Contours: Model, Units: nanoTesla]
""".strip()
# Place text
fig.text(0, 1.05, description, va="top", fontfamily="monospace", fontsize=12);